In [ ]:
def decode_routes(permutation, demands, Q):
    """
    permutation: lista de clientes [c1, c2, ..., cn]
    demands: dict {cliente -> demanda}
    Q: capacidad del vehículo
    return: lista de rutas, cada ruta es [depot, ..., depot]
    """
    routes = []
    current_route = []
    current_load = 0

    for client in permutation:
        d = demands[client]
        if current_load + d <= Q:
            current_route.append(client)
            current_load += d
        else:
            routes.append(current_route)
            current_route = [client]
            current_load = d

    if current_route:
        routes.append(current_route)

    # añadir depósito al inicio y final de cada ruta
    routes_with_depot = []
    for r in routes:
        routes_with_depot.append([0] + r + [0])

    return routes_with_depot
def evaluate_solution(permutation, instance):
    """
    instance tendrá todo: distancias, demandas, Q, parámetros de costos, ...
    """
    routes = decode_routes(permutation, instance.demands, instance.Q)
    total_distance = 0.0
    total_time = 0.0
    total_fuel_cost = 0.0

    for route in routes:
        for i in range(len(route) - 1):
            a = route[i]
            b = route[i+1]
            dist = instance.distance_matrix[a][b]
            total_distance += dist

            time = dist / instance.speed  # tiempo = distancia / velocidad cte
            total_time += time

            fuel_used = dist / instance.fuel_efficiency  # km / (km/l) = l
            total_fuel_cost += fuel_used * instance.fuel_price

    num_vehicles = len(routes)
    fixed_cost = num_vehicles * instance.C_fixed
    distance_cost = total_distance * instance.C_dist
    time_cost = total_time * instance.C_time

    total_cost = fixed_cost + distance_cost + time_cost + total_fuel_cost

    return total_cost, routes

import random

def swap_mutation(perm):
    i, j = random.sample(range(len(perm)), 2)
    perm[i], perm[j] = perm[j], perm[i]
    return perm

def ox_crossover(parent1, parent2):
    n = len(parent1)
    a, b = sorted(random.sample(range(n), 2))
    child = [None] * n

    child[a:b+1] = parent1[a:b+1]

    p2_idx = 0
    for i in range(n):
        if child[i] is None:
            while parent2[p2_idx] in child:
                p2_idx += 1
            child[i] = parent2[p2_idx]

    return child

import random
import numpy as np

class CVRPInstance:
    def __init__(self, distance_matrix, demands, Q,
                 C_fixed, C_dist, C_time,
                 fuel_price, fuel_efficiency,
                 speed=1.0):
        self.distance_matrix = distance_matrix
        self.demands = demands          # dict {client_id -> demanda}
        self.Q = Q
        self.C_fixed = C_fixed
        self.C_dist = C_dist
        self.C_time = C_time
        self.fuel_price = fuel_price
        self.fuel_efficiency = fuel_efficiency
        self.speed = speed

        self.clients = sorted([c for c in demands.keys() if c != 0])


def decode_routes(permutation, demands, Q):
    routes = []
    current_route = []
    current_load = 0

    for client in permutation:
        d = demands[client]
        if current_load + d <= Q:
            current_route.append(client)
            current_load += d
        else:
            routes.append(current_route)
            current_route = [client]
            current_load = d

    if current_route:
        routes.append(current_route)

    routes_with_depot = []
    for r in routes:
        routes_with_depot.append([0] + r + [0])

    return routes_with_depot


def evaluate_solution(permutation, instance: CVRPInstance):
    routes = decode_routes(permutation, instance.demands, instance.Q)
    total_distance = 0.0
    total_time = 0.0
    total_fuel_cost = 0.0

    for route in routes:
        for i in range(len(route) - 1):
            a = route[i]
            b = route[i+1]
            dist = instance.distance_matrix[a][b]
            total_distance += dist

            time = dist / instance.speed
            total_time += time

            fuel_used = dist / instance.fuel_efficiency
            total_fuel_cost += fuel_used * instance.fuel_price

    num_vehicles = len(routes)
    fixed_cost = num_vehicles * instance.C_fixed
    distance_cost = total_distance * instance.C_dist
    time_cost = total_time * instance.C_time

    total_cost = fixed_cost + distance_cost + time_cost + total_fuel_cost

    return total_cost, routes


def init_population(instance, pop_size):
    population = []
    for _ in range(pop_size):
        perm = instance.clients.copy()
        random.shuffle(perm)
        population.append(perm)
    return population


def tournament_selection(population, fitnesses, k=3):
    idxs = random.sample(range(len(population)), k)
    best = min(idxs, key=lambda i: fitnesses[i])
    return population[best][:]  


def swap_mutation(perm, p_mut):
    perm = perm[:]  
    if random.random() < p_mut:
        i, j = random.sample(range(len(perm)), 2)
        perm[i], perm[j] = perm[j], perm[i]
    return perm


def ox_crossover(parent1, parent2, p_crossover):
    if random.random() > p_crossover:
        return parent1[:], parent2[:]

    n = len(parent1)
    a, b = sorted(random.sample(range(n), 2))
    child1 = [None] * n
    child2 = [None] * n

    # segmento de parent1 en child1; de parent2 en child2
    child1[a:b+1] = parent1[a:b+1]
    child2[a:b+1] = parent2[a:b+1]

    # fill child1 con parent2
    p2_idx = 0
    for i in range(n):
        if child1[i] is None:
            while parent2[p2_idx] in child1:
                p2_idx += 1
            child1[i] = parent2[p2_idx]

    # fill child2 con parent1
    p1_idx = 0
    for i in range(n):
        if child2[i] is None:
            while parent1[p1_idx] in child2:
                p1_idx += 1
            child2[i] = parent1[p1_idx]

    return child1, child2


def run_ga(instance: CVRPInstance,
           pop_size=50,
           generations=200,
           p_crossover=0.9,
           p_mut=0.2,
           tournament_k=3,
           seed=None):

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    population = init_population(instance, pop_size)

    # evaluar población inicial
    fitnesses = []
    for perm in population:
        cost, _ = evaluate_solution(perm, instance)
        fitnesses.append(cost)

    best_cost = min(fitnesses)
    best_perm = population[int(np.argmin(fitnesses))][:]
    history_best = [best_cost]

    for gen in range(generations):
        new_population = []

        # elitismo: conservar el mejor
        new_population.append(best_perm[:])

        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses, k=tournament_k)
            parent2 = tournament_selection(population, fitnesses, k=tournament_k)

            child1, child2 = ox_crossover(parent1, parent2, p_crossover)
            child1 = swap_mutation(child1, p_mut)
            child2 = swap_mutation(child2, p_mut)

            new_population.append(child1)
            if len(new_population) < pop_size:
                new_population.append(child2)

        population = new_population

        # reevaluar
        fitnesses = []
        for perm in population:
            cost, _ = evaluate_solution(perm, instance)
            fitnesses.append(cost)

        gen_best_cost = min(fitnesses)
        gen_best_perm = population[int(np.argmin(fitnesses))][:]

        if gen_best_cost < best_cost:
            best_cost = gen_best_cost
            best_perm = gen_best_perm

        history_best.append(best_cost)

    best_routes = decode_routes(best_perm, instance.demands, instance.Q)
    return best_cost, best_routes, best_perm, history_best


### Implementación del GA base para el CVRP

En este bloque de código se implementa una primera versión de un Algoritmo Genético (GA) para resolver el problema de ruteo de vehículos capacitado (CVRP) usando una representación tipo *giant tour*.



#### 1. Clase CVRPInstance

Definimos la clase CVRPInstance para agrupar todos los datos del problema en un solo objeto:

- distance_matrix: matriz de distancias entre todos los nodos (depósito y clientes).
- demands: diccionario que mapea cada cliente a su demanda, demands[cliente].
- Q: capacidad máxima de cada vehículo.
- C_fixed: costo fijo por vehículo utilizado.
- C_dist: costo por unidad de distancia.
- C_time: costo por unidad de tiempo.
- fuel_price: precio del combustible.
- fuel_efficiency: rendimiento de combustible (por ejemplo, km/l).
- speed: velocidad del vehículo (para convertir distancia en tiempo).

La lista self.clients se construye como todos los nodos distintos del depósito, y será la base para generar las permutaciones (cromosomas) del GA.



#### 2. repressentación y decodificación: decode_routes

La representación genética que usamos es una permutación de todos los clientes:

$
\text{perm} = [c_1, c_2, \dots, c_n]
$

Esta representación no separa explícitamente las rutas de los vehículos. Para obtener las rutas reales, usamos la función decode_routes(permutation, demands, Q):

1. Recorremos la permutación en orden.
2. Mantenemos una ruta actual y su carga acumulada.
3. Para cada cliente client:
   - Si su demanda $ d_{client} $ cabe en la ruta actual sin sobrepasar la capacidad $ Q $, lo agregamos.
   - Si no cabe, cerramos la ruta actual y empezamos una nueva.
4. Al final, añadimos el depósito al inicio y al final de cada ruta, de modo que cada ruta tenga la forma:

$
[0, c_{i_1}, c_{i_2}, \dots, c_{i_k}, 0]
$

Es decir, todas las rutas empiezan y terminan en el depósito y respetan la capacidad del vehículo.



#### 3. evaluación de una solución: evaluate_solution

La función evaluate_solution(permutation, instance):

1. Decodifica la permutación en una lista de rutas factibles mediante decode_routes.
2. Recorre cada ruta y acumula:
   - Distancia total recorrida total_distance.
   - Tiempo total total_time, calculado como:

$
\text{time} = \frac{\text{distancia}}{\text{speed}}
$

   - Costo de combustible total_fuel_cost, usando:

$
\text{combustible usado} = \frac{\text{distancia}}{\text{fuel\_efficiency}}, \quad
\text{costo combustible} = \text{combustible usado} \times \text{fuel\_price}
$

3. Calcula el número de vehículos utilizados como el número de rutas generadas.
4. Calcula el costo total como:
$
\text{TotalCost} = (\text{num\_vehicles} \times C_{\text{fixed}})\ +\ (\text{total\_distance} \times C_{\text{dist}})\ +\ (\text{total\_time} \times C_{\text{time}})\ +\ \text{total\_fuel\_cost}
$

La función retorna:

- total_cost: valor de la función objetivo de la solución.
- routes: las rutas construidas (lista de listas con depósito al inicio y al final).



#### población inicial: init_population

La función init_population(instance, pop_size) genera la población inicial del GA:

- Toma la lista de clientes instance.clients.
- Genera pop_size permutaciones aleatorias mediante random.shuffle.
- Cada permutación es un cromosoma que representa una solución candidata.



####  tournament_selection

La selección de padres se realiza mediante torneo:

1. Se eligen k individuos al azar de la población.
2. Se compara el valor de fitness (en este caso, el costo total: menor es mejor).
3. Se devuelve una copia del mejor individuo entre los seleccionados.

Este método favorece soluciones buenas, pero mantiene diversidad gracias a la aleatoriedad en la elección de los participantes del torneo.



#### crossover: ox_crossover

Para recombinar dos padres usamos un cruce de tipo Order Crossover (OX) adaptado a permutaciones:

1. Se elige un segmento aleatorio [a, b].
2. Ese segmento se copia directamente del padre 1 al hijo 1, y del padre 2 al hijo 2.
3. Los huecos restantes en cada hijo se rellenan respetando el orden del otro padre, sin repetir clientes.

Con esto se generan dos hijos que son permutaciones válidas de los clientes.

El crossover se aplica con probabilidad p_crossover. Si no se aplica, los hijos son copias de los padres.



#### mutación: swap_mutation

La mutación se implementa como un swap:

1. Con probabilidad p_mut elegimos dos posiciones al azar en la permutación.
2. Intercambiamos los clientes de esas dos posiciones.

La mutación ayuda a mantener diversidad en la población y evitar estancarse en óptimos locales.



#### documentacion principal del GA: run_ga

La función run_ga implementa el ciclo completo del GA:

1. Inicializa la población usando init_population.
2. Evalúa la población inicial usando evaluate_solution.
3. Guarda el mejor individuo encontrado (elitismo) y su costo.
4. Para cada generación:
   - Crea una nueva población:
     - Conserva el mejor individuo (elitismo).
     - Repite:
       - Selecciona dos padres por torneo.
       - Aplica crossover OX con probabilidad p_crossover.
       - Aplica mutación swap con probabilidad p_mut a cada hijo.
       - Agrega los hijos a la nueva población.
   - Evalúa la nueva población completa.
   - Actualiza el mejor individuo global si encuentra una solución mejor.
   - Guarda el historial del mejor costo encontrado en cada generación (history_best).

Al final, la función retorna:

- best_cost: mejor valor de la función objetivo encontrado.
- best_routes: rutas decodificadas de la mejor permutación.
- best_perm: la mejor permutación (cromosoma) encontrada.
- history_best: lista con el mejor costo por generación (sirve para graficar la convergencia del GA).

Este GA repreenta la plantilla sobre la cual se conectan los datos específicos de cada instancia (Caso Base, Caso 2 y Caso 3) y el análisis comparativo con Pyomo.
